In [5]:
!pip install inputs

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from inputs import get_gamepad
from jetracer.nvidia_racecar import NvidiaRacecar
import requests

In [2]:
car = NvidiaRacecar()

In [3]:
car.steering = 0
car.steering_offset = 0
car.throttle_gain = 0.20

In [8]:
def update_display():
    info = '\n'.join([
        f"th_gain: {car.throttle_gain:.2f}",
        f"st_offs: {car.steering_offset:.2f}",
    ])
    requests.get("http://localhost:8000/text/" + info)

In [9]:
def reset_display():
    requests.get("http://localhost:8000/stats/on")

# Instrucciones para controlar desde el mando:
- Joystick izquierdo: **acelera / frena**
- Joystick derecho: **dirección**
- X / B: **offset dirección**. En la pantalla del coche se muestra el offset actual.
- Y / A: **velocidad máxima** (ojo, a partir de 0.2 corre **MUCHO**). En la pantalla del coche se muestra la velocidad máxima actual.
- Gatillos izquierdo y derecho a la vez: **finaliza el bucle de control y detiene el coche** (finaliza el control remoto)

In [10]:
right_trigger = left_trigger = 0


while not (right_trigger and left_trigger):
    events = get_gamepad()
    for event in events:
        if event.ev_type == 'Absolute':
            if event.code == 'ABS_Y':
                car.throttle = -(event.state - 127.5) / 127.5
                #print(f'Y={event.state}')
            if event.code == 'ABS_Z':
                car.steering = (event.state - 127.5) / 127.5
                #print(f'Z={event.state}')
        elif event.ev_type == 'Key' and event.code == 'BTN_TR2':
            right_trigger = event.state
        elif event.ev_type == 'Key' and event.code == 'BTN_TL2':
            left_trigger = event.state
        elif event.ev_type == 'Key' and event.state == 1:
            if event.code == 'BTN_WEST': # Y / UP
                car.throttle_gain = min(1.0, car.throttle_gain + 0.05)
                update_display()
            if event.code == 'BTN_SOUTH': # A / DOWN
                car.throttle_gain = max(0.0, car.throttle_gain - 0.05)
                update_display()
            if event.code == 'BTN_EAST': # B / RIGHT
                car.steering_offset = max(-0.3, car.steering_offset - 0.05)
                car.steering = 0.1
                car.steering = 0
                update_display()
            if event.code == 'BTN_NORTH': # X / LEFT
                car.steering_offset = min(0.3, car.steering_offset + 0.05)
                car.steering = 0.1
                car.steering = 0
                update_display()
                
            #print(f"throttle_gain={car.throttle_gain:.1f}")
        #print(event.ev_type, event.code, event.state)
        
car.throttle = 0
reset_display()